In [1]:
#PATHS
#Daha kolay bir sekilde yazmak ve okumak icin pathler bu sekilde olusturuldu.
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/modell_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/modell_ssd_mobnet/'

In [4]:
# Tensorflow Records
#Verileri Tensowflowun anlayabilecegi sekle donusturup egitim icin hazir hale getiriyor.

!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}


Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


In [4]:
# Githubdan Tensorflow nesne tespiti icin gerekli olan belgeler indiriliyor.
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:  61% (1377/2250)
Updating files:  62% (1395/2250)
Updating files:  63% (1418/2250)
Updating files:  64% (1440/2250)
Updating files:  65% (1463/2250)
Updating files:  66% (1485/2250)
Updating files:  67% (1508/2250)
Updating files:  68% (1530/2250)
Updating files:  69% (1553/2250)
Updating files:  70% (1575/2250)
Updating files:  71% (1598/2250)
Updating files:  72% (1620/2250)
Updating files:  73% (1643/2250)
Updating files:  74% (1665/2250)
Updating files:  75% (1688/2250)
Updating files:  76% (1710/2250)
Updating files:  77% (1733/2250)
Updating files:  78% (1755/2250)
Updating files:  79% (1778/2250)
Updating files:  80% (1800/2250)
Updating files:  81% (1823/2250)
Updating files:  82% (1845/2250)
Updating files:  83% (1868/2250)
Updating files:  84% (1890/2250)
Updating files:  85% (1913/2250)
Updating files:  86% (1935/2250)
Updating files:  87% (1958/2250)
Updating files:  88% (1980/2250)
Updating files:  89% (2003/2250)
Updating files:  9

In [2]:
# Tensorflow Model Zoodan temin edilen modelin pipeline konfigurasyon dosyasi
#projede kullanilacak fonksiyonlara gore uygun hale getirilecek
MY_MODEL_NAME = 'modell_ssd_mobnet' 

In [3]:
# Tensorflow kutuphanesi import edildi ve konfigurasyon dosyasinda degisiklik yapmak
#icin birkac kutuphaneden fonksiyonlar import edildi
import tensorflow as tf
from object_detection.utils import config_util
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

In [7]:
#Konfigurasyon dosyasini acmak icin pathi olusturuldu ve dosya okundu
CONFIG_PATH = MODEL_PATH+'/'+MY_MODEL_NAME+'/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [9]:

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [10]:
#Buradaki parametreler projeye uygun bir sekilde degistirildi ornegin ilk satirdaki class sayisi 19 olarak degistirildi.
pipeline_config.model.ssd.num_classes = 19
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [11]:
#Yapilan degisiklikler kaydediliyor ve dosyaya f.write ile yaziliyor.
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)  

In [13]:
# Komut penceresinde cd ile calisma klasorune gidilir ve alttaki kod yapistirilip runlanir. 
#Burada test sonuclarina gore num_train_steps parametresi degistirilebilir. Performans testlerinden sonra en iyi performansi 40000 olarak karar verdim.
#Azi yeterince iyi tespit etmiyo fazlasi ise ezberlemeye yol aciyor.
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=40000""".format(APIMODEL_PATH, MODEL_PATH,MY_MODEL_NAME,MODEL_PATH,MY_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/modell_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/modell_ssd_mobnet/pipeline.config --num_train_steps=40000


In [4]:
#Bundan sonrasi bilgisayarda test etmek icindir.
#Egitilen model checkpoint dosyalari olusturur.
# Sistem dosyalari arasinda gezinmek icin os kutuphanesi, olusturulan checkpointten model insa etmek icin
#model_builder ve label_map_util kutuphaneleri importlandi.
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [5]:
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)#Duzenlenen konfigurasyon dosyasini yukler
my_model = model_builder.build(model_config=configs['model'], is_training=False)#Duzenlenen konfigurasyon dosyasini kullanarak model olusturur.

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=my_model)#Tahminlerde kullanmasi icin en gelismis chechkpoint secilir.
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-41')).expect_partial()

@tf.function
def detector(image):
    image, shapes = my_model.preprocess(image)#input fotografini modele girmesi icin resize gibi islemlerle uygun hale getirir.
    prediction_dict = my_model.predict(image, shapes)# Tahmin olusturulur
    detections = my_model.postprocess(prediction_dict, shapes)#Output verilerini anlasilmasi daha kolay bir hal olan dictionarye donusturur.
    return detections

In [6]:
# Kamera icin opencv numpy importlanir.
import cv2 
import numpy as np
import tensorflow as tf

In [7]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')#Kategori indexi olusturuluyor. Bu tahmini label maple esleyecek olan ve kullanilacak degisken.

In [8]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [9]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)# Kameranin gordugu framei tensore cevirir.
    detections = detector(input_tensor)#detections tahminlerin isimlerini koordinatlarini ve skorlarini tutar.
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # classes integera donusturuldu.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+1,#kategori indexi ile uyusmasi icin +1 konuldu.
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.7,
                agnostic_mode=False)# Elin etrafina kutu ciziyor ve skoru ve ismini yazdiriyor

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if detections['detection_scores'][0] >= 0.7:
        command=category_index[detections['detection_classes'][0]+1]['name']
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break